# Processing Pipelines

In questa parte andremo a vedere cosa succede sotto il cofano quando si utilizza una pipeline SpaCy.

<br>

![pipeline](https://course.spacy.io/pipeline.png)

<br>

Una pipeline è una serie di funzioni da applicare ad un doc per aggiungere attributi come POS-tag, dipendenze, ...

Come si vede nell'immagine al momento la pipeline basica (SpaCy ha molte componenti da utilizzare in una pipeline) ha un testo in input poi in sequenza applica:
* tokenizer: suddivide il testo in token
* tagger: il POS tagging imposta i `token.tag` e `token.pos` (es. *"Apple is based in California"*, *Apple*: **POS**=PROPN e **TAG**=NNP) la differenza è che `.pos` e `.pos_` si riferiscono all'**universal POS tags** mentre `.tag` e `.tag_` al **fine-gradied POS tags**.
* parser: aggiunge gli attributi `token.head` e `token.dep` individua i token base nella frase e le dipendenze
* NER: indica le entità nel testo e aggiunge l'attributo `.ents` (come entità definiamo ad esempio COMPANY, MONEY, ... o altre regole che possiamo aggiungere manualmente con il *matching*)

e restituisce un oggetto `Doc`.

Tutte le pipeline che possiamo caricare in SpaCy hanno diversi file/componenti e un file di configurazione *config.cfg*, il config definisce le componenti base della pipeline e la lingua. Anche componenti predittive come ad esempio *tok2vec* preaddestrato (che serve per creare l'embedding del token) sono caricate nella pipeline.

<br>

![pipeline_comp](https://course.spacy.io/package_meta.png)

<br>

> altra componente della pipeline molto usata è il category labels, aggiunge l'attributo `.cats`, questo non si applica al token ma all'intero testo

Per vedere le componenti della pipeline bisogna usare `nlp.pipe_names`.

In [3]:
import spacy

nlp = spacy.load("en_core_web_sm")

print(nlp.pipe_names)

# ritorna la lista [nome funzione, components]
# print(nlp.pipeline)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


## Custom pipeline components

Le componenti custom permettono di inserire nuovi step personalizzati nella pipeline.

A seguito della tokenizzazione la pipeline SpaCy esegue le componenti in sequenza, a queste possono essere aggiunte quelle create da noi che verranno eseguite autometicamente quando si richiama la pipeline. Questi elementi sono molto utili per espandere i metadati del documento o per aggiungere attributi nella NER.

La pipeline è una funzione (o un *callable*) che prende un testo in input lo modifica e ce lo ritorna per essere processato al prossimo step.

Per aggiungere degli step bisogna usare il decorator `@Language.component` prima della funzione per creare lo step custom. A seguito della funzione bisogna aggiungere lo step alla pipeline con `nlp.add_pipe`.

Si può anche specificare in che posizione della pipeline deve essere inserito questo componente.

| Argomento | Descrizione | Esempio |
|:---:|:---:|:---|
| last | Se `True` aggiungi alla fine | `nlp.add_pipe("component", last=True)` |
| first | Se `True` aggiungi all0inizio | `nlp.add_pipe("component", first=True)` |
| before | Aggiunge prima del passo indicato | `nlp.add_pipe("component", before="ner")` |
| after | Aggiunge di seguito al passo indicato | `nlp.add_pipe("component", after="tagging")` |

Quando si crea la funzione per la componente custom è fondamentale non dimenticare di ritornare il `Doc` in output, così può essere l'input dello step successivo. Di seguito un esempio di un componente aggiuntivo che inserito all'inizio della pipeline ritorna la lunghezza del documento.

In [5]:
from spacy.language import Language

# Create the nlp object
nlp = spacy.load("en_core_web_sm")

# Define a custom component
@Language.component("custom_component")
def custom_component_function(doc):
    # Print the doc's length
    print("Doc length:", len(doc))
    # Return the doc object
    return doc

# Add the component first in the pipeline
nlp.add_pipe("custom_component", first=True)

# Print the pipeline component names
print("Pipeline:", nlp.pipe_names)

Pipeline: ['custom_component', 'tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [12]:
doc = nlp("Hello world! This is an example")

for token in doc:
    print(token.text)

Doc length: 7
Hello
world
!
This
is
an
example


Di seguito un esempio più complesso dove viene aggiunto uno step della pipeline dopo la NER per esolare solo alcune parole etichettate come *"ANIMAL"*.

In [14]:
import spacy
from spacy.language import Language
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span

nlp = spacy.load("en_core_web_sm")
animals = ["Golden Retriever", "cat", "turtle", "Rattus norvegicus"]
animal_patterns = list(nlp.pipe(animals))
print("animal_patterns:", animal_patterns)
matcher = PhraseMatcher(nlp.vocab)
matcher.add("ANIMAL", animal_patterns)

# Define the custom component
@Language.component("animal_component")
def animal_component_function(doc):
    # Apply the matcher to the doc
    matches = matcher(doc)
    # Create a Span for each match and assign the label "ANIMAL"
    spans = [Span(doc, start, end, label="ANIMAL") for match_id, start, end in matches]
    # Overwrite the doc.ents with the matched spans
    doc.ents = spans
    return doc


# Add the component to the pipeline after the "ner" component
nlp.add_pipe("animal_component", after="ner")
print(nlp.pipe_names)

# Process the text and print the text and label for the doc.ents
doc = nlp("I have a cat and a Golden Retriever")
print([(ent.text, ent.label_) for ent in doc.ents])

animal_patterns: [Golden Retriever, cat, turtle, Rattus norvegicus]
['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner', 'animal_component']
[('cat', 'ANIMAL'), ('Golden Retriever', 'ANIMAL')]
